In [ ]:
!pip install tensorflow==2.3.0
!pip install gym
!pip install keras
!pip install keras-rl2

In [ ]:
import numpy as np
from gym import Env
from gym.spaces import Box, Discrete
import random

In [ ]:
class CustomEnv(Env):
  def __init__(self):
    self.action_space = Discrete(3)
    self.observation_space = Box(low=np.array([0]), high=np.array([100]))
    self.state = 38 + random.randint(-3,3)
    self.shower_length = 60
  def step(self, action):
    self.state += action -1 
    self.shower_length -= 1 
    
    # Calculating the reward
    if self.state >=37 and self.state <=39: 
        reward =1 
    else: 
        reward = -1 
    
    # Checking if shower is done
    if self.shower_length <= 0: 
        done = True
    else:
        done = False
    
    # Setting the placeholder for info
    info = {}
    
    # Returning the step information
    return self.state, reward, done, info
  def reset(self):
    self.state = 38 + random.randint(-3,3)
    self.shower_length = 60 
    return self.state


In [ ]:
env = CustomEnv()
episodes = 20 #20 shower episodes
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

Episode:1 Score:-16
Episode:2 Score:-60
Episode:3 Score:-40
Episode:4 Score:-60
Episode:5 Score:-60
Episode:6 Score:-56
Episode:7 Score:-24
Episode:8 Score:-8
Episode:9 Score:-26
Episode:10 Score:-22
Episode:11 Score:-16
Episode:12 Score:20
Episode:13 Score:-46
Episode:14 Score:-24
Episode:15 Score:-26
Episode:16 Score:26
Episode:17 Score:-12
Episode:18 Score:-52
Episode:19 Score:-60
Episode:20 Score:-2


/usr/local/lib/python3.7/dist-packages/gym/spaces/box.py:128: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:513: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  np.object,
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:521: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  np.bool,
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/tensor_util.py:107: DeprecationWarning: `np.object` is a deprecated ali

In [ ]:
states = env.observation_space.shape
actions = env.action_space.n

In [ ]:
def build_model(states, actions):
    model = Sequential()    
    model.add(Dense(24, activation='relu', input_shape=states))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [ ]:
model = build_model(states, actions)
# del model

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 24)                48        
_________________________________________________________________
dense_7 (Dense)              (None, 24)                600       
_________________________________________________________________
dense_8 (Dense)              (None, 3)                 75        
Total params: 723
Trainable params: 723
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [ ]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [ ]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=60000, visualize=False, verbose=1)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training_utils.py:1446: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  if isinstance(sample_weight_mode, collections.Mapping):
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Training for 60000 steps ...
Interval 1 (0 steps performed)
    1/10000 [..............................] - ETA: 10:46 - reward: -1.0000

/usr/local/lib/python3.7/dist-packages/rl/memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')
/usr/local/lib/python3.7/dist-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 10 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)


   19/10000 [..............................] - ETA: 5:35 - reward: -1.0000

/usr/local/lib/python3.7/dist-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 11 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
/usr/local/lib/python3.7/dist-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 12 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
/usr/local/lib/python3.7/dist-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 13 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
/usr/local/lib/python3.7/dist-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 14 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
/usr/local/lib/python3.7/dist-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 15 + 1) instead
  batch_idxs

   36/10000 [..............................] - ETA: 4:10 - reward: -1.0000

/usr/local/lib/python3.7/dist-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 22 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
/usr/local/lib/python3.7/dist-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 23 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
/usr/local/lib/python3.7/dist-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 24 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
/usr/local/lib/python3.7/dist-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 25 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
/usr/local/lib/python3.7/dist-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 26 + 1) instead
  batch_idxs

10000/10000 [==============================] - 86s 9ms/step - reward: -0.5966
166 episodes - episode_reward: -35.699 [-60.000, 34.000] - loss: 0.994 - mae: 5.576 - mean_q: -6.567

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 85s 8ms/step - reward: -0.5788
167 episodes - episode_reward: -34.922 [-60.000, 36.000] - loss: 1.841 - mae: 9.837 - mean_q: -14.062

Interval 3 (20000 steps performed)
  497/10000 [>.............................] - ETA: 1:17 - reward: -0.5252done, took 174.287 seconds


In [ ]:
results = dqn.test(env, nb_episodes=200, visualize=True)
print(np.mean(results.history['episode_reward']))

Testing for 150 episodes ...
Episode 1: reward: 60.000, steps: 60
Episode 2: reward: -60.000, steps: 60
Episode 3: reward: -60.000, steps: 60
Episode 4: reward: 60.000, steps: 60
Episode 5: reward: 60.000, steps: 60
Episode 6: reward: -60.000, steps: 60
Episode 7: reward: -60.000, steps: 60
Episode 8: reward: -60.000, steps: 60
Episode 9: reward: -60.000, steps: 60
Episode 10: reward: -60.000, steps: 60
Episode 11: reward: -60.000, steps: 60
Episode 12: reward: 60.000, steps: 60
Episode 13: reward: 60.000, steps: 60
Episode 14: reward: -60.000, steps: 60
Episode 15: reward: 60.000, steps: 60
Episode 16: reward: 60.000, steps: 60
Episode 17: reward: -60.000, steps: 60
Episode 18: reward: 60.000, steps: 60
Episode 19: reward: -60.000, steps: 60
Episode 20: reward: -60.000, steps: 60
Episode 21: reward: -60.000, steps: 60
Episode 22: reward: 60.000, steps: 60
Episode 23: reward: 60.000, steps: 60
Episode 24: reward: -60.000, steps: 60
Episode 25: reward: 60.000, steps: 60
Episode 26: rewa